# Simrunner - Example use with Tuflow
Author: Tom Norman  
Version 0.1.0 (Prerelease)

__Disclaimer__: 
This package, or any of the work associated with simrunner has absolutely no affiliation with Tuflow or BMT. 

A working example of using Simrunner to call the demonstration examples of Tuflow, downloaded [here](https://downloads.tuflow.com/TUFLOW/Wiki_Example_Models/TUFLOW_Example_Model_Dataset.zip). The demostration files are all lumped together and are not the prefered way of setting out your project. This example demostrates the flexibility of Simrunner when you need it.

The batch file we are replicating is below:

```Batch
set EXE_iSP=..\exe\2023-03-AC\TUFLOW_iSP_w64.exe
set EXE_iDP=..\exe\2023-03-AC\TUFLOW_iDP_w64.exe

start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg EG16_~s1~_001.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01 EG16_~s1~_001.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg  -s2  5m EG16_~s1~_~s2~_002.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01  -s2  5m EG16_~s1~_~s2~_002.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg  -s2  2.5m EG16_~s1~_~s2~_002.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01  -s2  2.5m EG16_~s1~_~s2~_002.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg  -s2  5m EG16_~s1~_~s2~_003.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01  -s2  5m EG16_~s1~_~s2~_003.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  Q100 EG16_~e1~_004.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  QPMF EG16_~e1~_004.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  Q100 -e2 2hr EG16_~e1~_~e2~_005.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  Q100 -e2 4hr EG16_~e1~_~e2~_005.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  QPMF -e2 2hr EG16_~e1~_~e2~_005.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -el  QPMF -e2 4hr EG16_~e1~_~e2~_005.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg  -s2  5m  -el  Q100 -e2  2hr EG16_~s1~_~s2~_~e1~_~e2~_006.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01  -s2  5m  -el  Q100 -e2  4hr EG16_~s1~_~s2~_~e1~_~e2~_006.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  Exg  -s2  5m  -el  QPMF -e2  2hr EG16_~s1~_~s2~_~e1~_~e2~_006.tcf
start "TUFLOW" /wait   %EXE_iSP% -b  -sl  D01  -s2  5m  -el  QPMF -e2  4hr EG16_~s1~_~s2~_~e1~_~e2~_006.tcf
```

Import Simrunner to so we can use it. 

In [9]:
import simrunner.tuflow as tfr

Set up the parameter files.

In [10]:
parameters: tfr.Parameters = tfr.Parameters(
    exec_path = r"..\tests\data\tuflow\examples\raw\exe",   # The path to the TUFLOW executable.
    root = r"..\tests\data\tuflow\examples\raw\runs",       # The root of the model, location of the .tcf file.
    version = r"2023-03-AC",                                # The version of the engine.
    engine = r"SP",                                         # Single precision.
    flags = ['-t'],                                         # Lets just test for now.
    async_runs = 2,                                         # Run two models at once.
    stdout = r'..\trash\stdout',                            # In a location away from the source code.
    group = r"EG16"                                         # Only focusing on EG16.
)

Create a dictionary of the runs you want to stage. 

In [11]:
runs = {
    "001": [
        tfr.Run(s1="Exg"),
        tfr.Run(s1="D01")
    ],
    "002": [
        tfr.Run(s1="Exg", s2="5m"),
        tfr.Run(s1="D01", s2="5m"),
        tfr.Run(s1="Exg", s2="2.5m"),
        tfr.Run(s1="D01", s2="2.5m")
    ],
    "003": [
        tfr.Run(s1="Exg", s2="5m"),
        tfr.Run(s1="D01", s2="5m")
    ],
    "004": [
        tfr.Run(e1="Q100"),
        tfr.Run(e1="QPMF")
    ],
    "005": [
        tfr.Run(e1="Q100", e2="2hr"),
        tfr.Run(e1="Q100", e2="4hr"),
        tfr.Run(e1="QPMF", e2="2hr"),
        tfr.Run(e1="QPMF", e2="4hr"),
    ],
    "006": [
        tfr.Run(s1="Exg", s2="5m", e1="Q100", e2="2hr"),
        tfr.Run(s1="D01", s2="5m", e1="Q100", e2="4hr"),
        tfr.Run(s1="Exg", s2="5m", e1="QPMF", e2="2hr"),
        tfr.Run(s1="D01", s2="5m", e1="QPMF", e2="4hr"),
    ]
}

Create the runner.

In [12]:
runner = tfr.Runner(parameters)

Stage the above run in the runner.

In [13]:
def ar(num):
    """Returns a list of the run arguments for a set of runs"""
    return list(runs[num][0].get_args().keys())

parameters.run_args = ar('001')     
""" If the folder is structured correctly, setting run_args ahead of time is not needed. 
This can get you in trouble as it is then possible to run the wrong set of runs.
When the wrong arguments are encounter the batch may stop running. This is why during 
staging the runs are checked to make sure their arguments are correct."""

runner.stage(
    [run for run in runs["001"]]
)

parameters.run_args = ar('002')
runner.stage(
    [run for run in runs["002"]]
)

parameters.run_args = ar("003")
runner.stage(
    [run for run in runs["003"]]
)

parameters.run_args = ar('004')
runner.stage(
    [run for run in runs["004"]]
)

parameters.run_args = ar('005')
runner.stage(
    [run for run in runs["005"]]
)

parameters.run_args = ar('006')
runner.stage(
    [run for run in runs["006"]]
)

Lets check to see what has been staged. Notice the different sets of run arguments? We ideally dont want this.  

In [14]:
runner.get_runs()

[{'s1': 'Exg'},
 {'s1': 'D01'},
 {'s1': 'Exg', 's2': '5m'},
 {'s1': 'D01', 's2': '5m'},
 {'s1': 'Exg', 's2': '2.5m'},
 {'s1': 'D01', 's2': '2.5m'},
 {'e1': 'Q100'},
 {'e1': 'QPMF'},
 {'e1': 'Q100', 'e2': '2hr'},
 {'e1': 'Q100', 'e2': '4hr'},
 {'e1': 'QPMF', 'e2': '2hr'},
 {'e1': 'QPMF', 'e2': '4hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}]

To run, we have to subset the runs into the corresponding run groups. If we call run with say, run_number='001', the correct TCF will be call, but every set of runs will be called with it. Runs are not associated with TCFs! They are sets of arguments. Simrunner makes is easy to add, remove and query runs.

In [15]:
runner_001 = tfr.Runner(parameters, runner)
runner_001.remove_runs(runner[2:])
runner_001.get_runs()

[{'s1': 'Exg'}, {'s1': 'D01'}]

By calling run with run_number="001" we are asking Simrunner to evaluate the runs staged in runner_001 against the TCF with group id 'EG16' and run_number '001', there should only be one TCF that satisfies this, otherwise your on your own.

In [16]:
runner_001.run("001")

executing: {'s1': 'Exg'}_001
executing: {'s1': 'D01'}_001
---- ALL RUNS COMPLETE ----


We can repeat this paradim for the other run groups 002 to 006. But lets just run 006.

In [17]:
runner_006 = tfr.Runner(parameters, runner)
runner_006.remove_runs(runner[:-2])
runner_006.get_runs()

[{'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}]

Test first

In [18]:
parameters.flags = ['-t']
runner_006.run("006")

executing: {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'}_006
executing: {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}_006
---- ALL RUNS COMPLETE ----


Then run

In [19]:
parameters.flags = ['-b']
runner_006.run("006")

executing: {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'}_006
executing: {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}_006
---- ALL RUNS COMPLETE ----
